In [1]:
import sys
import json
import folium
import pandas as pd
import numpy as np
import pickle as pkl

from pyproj import Transformer
from shapely.geometry import shape, Point

sys.path.append('..')

from src import STORAGE_PATH
from src.visualization import convert_dict_list_to_counts_dataframe, create_smooth_choropleth_layers

### Load data

In [2]:
DATA_DIRECTORY = STORAGE_PATH / 'graphs'

with open(DATA_DIRECTORY / 'bike_accidents_graph.pkl', 'rb') as f:
    bike_accidents_graph = pkl.load(f)

with open(DATA_DIRECTORY / 'nonbike_accidents_graph.pkl', 'rb') as f:
    nonbike_accidents_graph = pkl.load(f)

In [3]:
with open(STORAGE_PATH / 'maps' / 'wroclaw-max.geojson', 'r') as f:
    wroclaw_map = json.load(f)

### Map accidents to regions

In [4]:
regions_accidents = {'bike' : {}, 'nonbike' : {}}
graphs = {'bike' : bike_accidents_graph, 'nonbike' : nonbike_accidents_graph}

for feature in wroclaw_map['features']:
    for acc_type in regions_accidents:
        regions_accidents[acc_type][feature['properties']['osiedle']] = []

in_proj = 'epsg:2177'
out_proj = 'epsg:4326'
transformer = Transformer.from_crs(in_proj, out_proj)
        
for acc_type in graphs:
    for node in graphs[acc_type]:
        node_t = transformer.transform(node[1], node[0])
        point = Point(node_t[1], node_t[0])
        point_in_any = False
        for feature in wroclaw_map['features']:
            polygon = shape(feature['geometry'])
            if polygon.contains(point):
                regions_accidents[acc_type][feature['properties']['osiedle']].append(node)
                break

### Maps per accident type

In [5]:
folium_maps = []
for acc_type in regions_accidents:
    folium_map = folium.Map(location=[51.1, 17], zoom_start=9)
    choropleth_df = pd.DataFrame(convert_dict_list_to_counts_dataframe(regions_accidents[acc_type]))
    folium.Choropleth(
        geo_data=wroclaw_map,
        data=choropleth_df,
        columns=['names', 'counts'],
        name="choropleth",
        key_on='properties.osiedle',
        fill_color="Reds",
        bins=9,
        fill_opacity=0.7,
        line_opacity=0.2,
        legend_name=f"Heatmap of {acc_type} accidents",
    ).add_to(folium_map)
    folium_maps.append(folium_map)

In [6]:
folium_maps[0]

In [7]:
folium_maps[1]

### Normalized difference

In [8]:
choropleth_df = pd.DataFrame(convert_dict_list_to_counts_dataframe(regions_accidents['bike'])).sort_values('names')
choropleth_df['nonbike_counts'] = pd.DataFrame(convert_dict_list_to_counts_dataframe(regions_accidents['nonbike'])).sort_values('names')['counts']
avg_multiplier = choropleth_df['counts'].sum() / choropleth_df['nonbike_counts'].sum()
choropleth_df['predbike_counts'] = choropleth_df['nonbike_counts'] * avg_multiplier
choropleth_df['diffs'] = choropleth_df['counts'] / choropleth_df['predbike_counts']

In [9]:

folium_map = folium.Map(location=[51.1, 17], zoom_start=9)
bins = np.array(range(9)) * max(choropleth_df['diffs'].max(), -choropleth_df['diffs'].min()) / 8
folium.Choropleth(
            geo_data=wroclaw_map,
            data=choropleth_df,
            columns=['names', 'diffs'],
            name="choropleth",
            key_on='properties.osiedle',
            fill_color="BuPu",
            bins=bins,
            fill_opacity=0.7,
            line_opacity=0.2,
            legend_name=f"Heatmap of {acc_type} accidents",
).add_to(folium_map)
folium_map

In [10]:
folium_map = folium.Map(location=[51.1, 17], zoom_start=9)
for layer in create_smooth_choropleth_layers(choropleth_df, 'names', 'diffs', wroclaw_map, lambda k: k['properties']['osiedle'], '#4040FF', '#FF4040', mid_color='#FFFFFF', force_mid=1):
    layer.add_to(folium_map)
folium_map